In [1]:
import cv2
import pytesseract
import numpy as np
import pandas as pd

# ==== CONFIG ====
# Update this path to the correct location of your image file
IMAGE_PATH = "original.jpg"
OUTPUT_CSV = "inhibition_zones.csv"

# โหลดภาพ
img = cv2.imread(IMAGE_PATH)
if img is None:
    raise FileNotFoundError(f"Could not load image at path: {IMAGE_PATH}")
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
gray = cv2.medianBlur(gray, 5)

# ==== 1. Detect disks (HoughCircles) ====
circles = cv2.HoughCircles(
    gray, cv2.HOUGH_GRADIENT, dp=1.2, minDist=40,
    param1=50, param2=30, minRadius=20, maxRadius=60
)

results = []

if circles is not None:
    circles = np.round(circles[0, :]).astype("int")

    for (x, y, r) in circles:
        # Extract ROI around disk
        roi = img[y-r:y+r, x-r:x+r]

        # ==== 2. OCR to read drug label ====
        roi_gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
        roi_gray = cv2.threshold(roi_gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
        text = pytesseract.image_to_string(roi_gray, config="--psm 8 -c tessedit_char_whitelist=ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789")
        drug_name = text.strip()

        # ==== 3. Find inhibition zone ====
        # Threshold + contour
        th = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                   cv2.THRESH_BINARY_INV, 51, 2)
        contours, _ = cv2.findContours(th, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        zone_diameter_px = None
        for cnt in contours:
            (zx, zy), zr = cv2.minEnclosingCircle(cnt)
            dist = np.sqrt((x - zx) ** 2 + (y - zy) ** 2)
            # เลือก contour ที่ครอบ disk
            if dist < r:
                continue
            if cv2.pointPolygonTest(cnt, (x, y), False) >= 0:
                zone_diameter_px = 2 * zr
                break

        if zone_diameter_px is not None:
            # ==== 4. Convert pixel → mm ====
            px_per_mm = (2*r) / 6.0   # disk diameter = 6 mm
            zone_mm = zone_diameter_px / px_per_mm

            results.append({
                "Drug": drug_name,
                "InhibitionZone_mm": round(zone_mm, 2)
            })

# ==== Save to CSV ====
df = pd.DataFrame(results)
df.to_csv(OUTPUT_CSV, index=False)

print("Saved:", OUTPUT_CSV)
print(df)


TesseractNotFoundError: tesseract is not installed or it's not in your PATH. See README file for more information.